# nPE: A Configurable Processing Engine
#### Verification | Version 0.3.1 | Updated 2018.7.24
___

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = """
C:\Users\RyanL\OneDrive\Research\SEAL\processing-engine/source/load-ivy.sc
"""

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.math.pow

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}


import scala.math.pow

## Parallel Register File

### Single Register File

#### Definition

In [71]:
// class RFInputs(datawidth: Int, addrwidth: Int) {
//     val wEnable  = Input (Bool())
//     val rEnable  = Input (Bool())
//     val wAddr    = Input (UInt(addrwidth.W))
//     val wData    = Input (SInt(datawidth.W))
//     val rAddrInt = Input (UInt(addrwidth.W))
//     val rAddrExt = Input (UInt(addrwidth.W))
// }

class RF (dataWidth: Int, addrWidth: Int) extends Module {
  
    val io = IO(new Bundle {
        val wEnable  = Input (Bool())
        val rEnable  = Input (Bool())
        val wAddr    = Input (UInt(addrWidth.W))
        val wData    = Input (SInt(dataWidth.W))
        val rAddrInt = Input (UInt(addrWidth.W))
        val rAddrExt = Input (UInt(addrWidth.W))
        val rDataInt = Output(SInt(dataWidth.W))
        val rDataExt = Output(SInt(dataWidth.W))
    })
    
    val registers  = RegInit(Vec(Seq.fill(pow(2, addrWidth).toInt) { 0.S(addrWidth.W) }))
    
    when(io.wEnable) {
        registers(io.wAddr) := io.wData
    }
    
    when(io.rEnable) {
        io.rDataInt := registers(io.rAddrInt)
        io.rDataExt := registers(io.rAddrExt)
    } .otherwise {
        io.rDataInt := 0.S
        io.rDataExt := 0.S
    }
}

defined class RF

#### Verification

In [93]:
Driver(() => new RF(8, 4)) {
    uut => new PeekPokeTester(uut) {
         
        poke(uut.io.wEnable, true)   
        poke(uut.io.rEnable, true)

        poke(uut.io.wAddr, 1)
        poke(uut.io.wData, 1)
        
        step(1)
        
        // Read
        poke(uut.io.rAddrInt, 1)
        expect(uut.io.rDataInt, 1)
        
        poke(uut.io.rAddrExt, 1)
        expect(uut.io.rDataExt, 1)
        
        // Write
        poke(uut.io.wAddr, 2)
        poke(uut.io.wData, 2)
        
        step(1)
        
        // Read
        poke(uut.io.rAddrInt, 1)
        expect(uut.io.rDataInt, 1)
        
        poke(uut.io.rAddrExt, 2)
        expect(uut.io.rDataExt, 2)
        
        // Write
        poke(uut.io.wAddr, 3)
        poke(uut.io.wData, 3)
        
        step(1)
        
        // Read
        poke(uut.io.rAddrInt, 1)
        expect(uut.io.rDataInt, 1)
        
        poke(uut.io.rAddrExt, 2)
        expect(uut.io.rDataExt, 2)
        
        poke(uut.io.rAddrInt, 3)
        expect(uut.io.rDataInt, 3)
    }
}

[info] [0.000] Elaborating design...
[info] [0.001] Done elaborating.
Total FIRRTL Compile Time: 15.4 ms
Total FIRRTL Compile Time: 11.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1532540505437
test cmd70WrapperHelperRF Success: 7 tests passed in 8 cycles taking 0.004849 seconds
[info] [0.004] RAN 3 CYCLES PASSED


res92: Boolean = true

### Putting them Together

#### Definition

In [149]:
class PRF(ports: Int, bypass: String, dataWidth: Int, addrWidth: Int) extends Module {
    
    require(List("None", "Soft", "Hard").contains(bypass))
    
    val io = IO(new Bundle {
        val wEnable  = Input (Vec(ports, Bool()))
        val rEnable  = Input (Vec(ports, Bool()))
        val wAddr    = Input (Vec(ports, UInt(addrWidth.W)))
        val wData    = Input (Vec(ports, SInt(dataWidth.W)))
        val rAddrInt = Input (Vec(ports, UInt(addrWidth.W)))
        val rAddrExt = Input (Vec(ports, UInt(addrWidth.W)))
        val rDataInt = Output(Vec(ports, SInt(dataWidth.W)))
        val rDataExt = Output(Vec(ports, SInt(dataWidth.W)))
        val bpSel    = if (bypass == "Soft") Some(Input(Vec(ports, Bool()))) else None
    })
    
    if (bypass == "None" || bypass == "Soft") {
        
        val rf = Seq.fill(ports){ Module(new RF(dataWidth, addrWidth)) }
        
        rf.zipWithIndex.map{ case (x: RF, i: Int) => {
            
            x.io.wEnable  := io.wEnable(i)
            x.io.rEnable  := io.rEnable(i)
            x.io.wAddr    := io.wAddr(i)
            x.io.wData    := io.wData(i)
            x.io.rAddrInt := io.rAddrInt(i)
            x.io.rAddrExt := io.rAddrExt(i)
            
            when (io.bpSel.getOrElse(Seq.fill(ports){ false.B })(i)) {
                io.rDataInt(i) := x.io.wData
                io.rDataExt(i) := x.io.wData
            } .otherwise {
                io.rDataInt(i) := x.io.rDataInt
                io.rDataExt(i) := x.io.rDataExt
            }
        }}
        
    } else if (bypass == "Hard") {
        io.rDataInt := io.wData
        io.rDataExt := io.wData
    }
}

defined class PRF

#### Verification

In [168]:
Driver(() => new PRF(2, "Soft", 8, 4)) {
    uut => new PeekPokeTester(uut) {
         
        poke(uut.io.wEnable(0), true)  
        poke(uut.io.wEnable(1), true) 
        poke(uut.io.rEnable(0), true)
        poke(uut.io.rEnable(1), true)
        poke(uut.io.bpSel.get(0), false)
        poke(uut.io.bpSel.get(1), false)

        poke(uut.io.wAddr(0), 1)
        poke(uut.io.wAddr(1), 1)
        poke(uut.io.wData(0), 1)
        poke(uut.io.wData(1), 1)
        
        step(1)
        
        // Read
        poke(uut.io.rAddrInt(0), 1)
        poke(uut.io.rAddrInt(1), 1)
        expect(uut.io.rDataInt(0), 1)
        expect(uut.io.rDataInt(1), 1)
        
        poke(uut.io.rAddrExt(0), 1)
        poke(uut.io.rAddrExt(1), 1)
        expect(uut.io.rDataExt(0), 1)
        expect(uut.io.rDataExt(1), 1)
        
        // Write
        poke(uut.io.wAddr(0), 2)
        poke(uut.io.wAddr(1), 2)
        poke(uut.io.wData(0), 2)
        poke(uut.io.wData(1), 2)
        
        step(1)
        
        // Read
        poke(uut.io.rAddrInt(0), 1)
        poke(uut.io.rAddrInt(1), 1)
        expect(uut.io.rDataInt(0), 1)
        expect(uut.io.rDataInt(1), 1)
        
        poke(uut.io.rAddrExt(0), 2)
        poke(uut.io.rAddrExt(1), 2)
        expect(uut.io.rDataExt(0), 2)
        expect(uut.io.rDataExt(1), 2)
        
        // Write
        poke(uut.io.wAddr(0), 3)
        poke(uut.io.wAddr(1), 3)
        poke(uut.io.wData(0), 3)
        poke(uut.io.wData(1), 3)
        
        step(1)
        
        // Read
        poke(uut.io.rAddrInt(0), 1)
        poke(uut.io.rAddrInt(1), 1)
        expect(uut.io.rDataInt(0), 1)
        expect(uut.io.rDataInt(1), 1)
        
        poke(uut.io.rAddrExt(0), 2)
        poke(uut.io.rAddrExt(1), 2)
        expect(uut.io.rDataExt(0), 2)
        expect(uut.io.rDataExt(1), 2)
        
        poke(uut.io.rAddrInt(0), 3)
        poke(uut.io.rAddrInt(1), 3)
        expect(uut.io.rDataInt(0), 3)
        expect(uut.io.rDataInt(1), 3)
        
        // Bypass
        poke(uut.io.bpSel.get(0), true)
        poke(uut.io.bpSel.get(1), false)
        poke(uut.io.wData(0), 10)
        poke(uut.io.wData(1), 10)
        expect(uut.io.rDataInt(0), 10)
        expect(uut.io.rDataInt(1), 3)
        expect(uut.io.rDataExt(0), 10)
        expect(uut.io.rDataExt(1), 2)
    }
}

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 20.2 ms
Total FIRRTL Compile Time: 20.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1532541556613
test cmd148WrapperHelperPRF Success: 18 tests passed in 8 cycles taking 0.018508 seconds
[info] [0.015] RAN 3 CYCLES PASSED


res167: Boolean = true

## Inner Product Unit

### Parallel Multiplier

In [7]:
class pMultiplier(width: Int, bitwidth: Int) extends Module {
    
    require(width >= 1, "Width must be at least one.")
    require(bitwidth >= 1, "Bitwidth must be at least one.")
    
    val io = IO(new Bundle {
        val in1 = Input (Vec(width, SInt(bitwidth.W)))
        val in2 = Input (Vec(width, SInt(bitwidth.W)))
        val out = Output(Vec(width, SInt(bitwidth.W)))
    })
    
    io.out := (io.in1 zip io.in2).map { case(a, b) => a * b }
}

defined class pMultiplier

In [8]:
Driver(() => new pMultiplier(4, 8)) {
    uut => new PeekPokeTester(uut) {
        poke(uut.io.in1(0), 1) 
        poke(uut.io.in2(0), 2)
        expect(uut.io.out(0), 2)
        
        poke(uut.io.in1(1), 3) 
        poke(uut.io.in2(1), 4)
        expect(uut.io.out(1), 12)
        
        poke(uut.io.in1(2), 5)
        poke(uut.io.in2(2), 6)
        expect(uut.io.out(2), 30)
        
        poke(uut.io.in1(3), 7)
        poke(uut.io.in2(3), 8)
        expect(uut.io.out(3), 56)
  }
}

[info] [0.000] Elaborating design...
[info] [0.018] Done elaborating.
Total FIRRTL Compile Time: 256.8 ms
Total FIRRTL Compile Time: 17.1 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1532537592109
test cmd6WrapperHelperpMultiplier Success: 4 tests passed in 5 cycles taking 0.020276 seconds
[info] [0.006] RAN 0 CYCLES PASSED


res7: Boolean = true

### Additive Reduction Tree

In [80]:
// Recursively creates a balanced syntax tree
def nonassocPairwiseReduce[A](xs: List[A], op: (A, A) => A): A = {
  xs match {
    case Nil => throw new IllegalArgumentException
    case List(singleElem) => singleElem
    case sthElse => {
      val grouped = sthElse.grouped(2).toList
      val pairwiseOpd = for (g <- grouped) yield {
        g match {
          case List(a, b) => op(a, b)
          case List(x) => x
        }
      }
      nonassocPairwiseReduce(pairwiseOpd, op)
    }
  }
}


class AdditiveRT(width: Int, bitwidth: Int) extends Module {

    require(width >= 1, "Width must be at least one.")
    require(bitwidth >= 1, "Bitwidth must be at least one.")
    
    val io = IO(new Bundle {
        val in  = Input (Vec(width, SInt(bitwidth.W)))
        val out = Output(SInt(bitwidth.W))
    })
    
    io.out := nonassocPairwiseReduce(io.in toList, (x: SInt, y: SInt) => x + y)
}

defined function nonassocPairwiseReduce
defined class AdditiveRT

In [71]:
println(getVerilog(new AdditiveRT(4, 4)))

[info] [0.000] Elaborating design...
[info] [0.015] Done elaborating.
Total FIRRTL Compile Time: 21.9 ms

module cmd69WrapperHelperAdditiveRT( // @[:@3.2]
  input        clock, // @[:@4.4]
  input        reset, // @[:@5.4]
  input  [3:0] io_in_0, // @[:@6.4]
  input  [3:0] io_in_1, // @[:@6.4]
  input  [3:0] io_in_2, // @[:@6.4]
  input  [3:0] io_in_3, // @[:@6.4]
  output [3:0] io_out // @[:@6.4]
);
  wire [4:0] _T_12; // @[cmd69.sc 29:76:@8.4]
  wire [3:0] _T_13; // @[cmd69.sc 29:76:@9.4]
  wire [3:0] _T_14; // @[cmd69.sc 29:76:@10.4]
  wire [4:0] _T_15; // @[cmd69.sc 29:76:@11.4]
  wire [3:0] _T_16; // @[cmd69.sc 29:76:@12.4]
  wire [3:0] _T_17; // @[cmd69.sc 29:76:@13.4]
  wire [4:0] _T_18; // @[cmd69.sc 29:76:@14.4]
  wire [3:0] _T_19; // @[cmd69.sc 29:76:@15.4]
  wire [3:0] _T_20; // @[cmd69.sc 29:76:@16.4]
  assign _T_12 = $signed(io_in_0) + $signed(io_in_1); // @[cmd69.sc 29:76:@8.4]
  assign _T_13 = _T_12[3:0]; // @[cmd69.sc 29:76:@9.4]
  assign _T_14 = $signed(_T_13); // @[cm

In [81]:
Driver(() => new AdditiveRT(4, 8)) {
  uut => new PeekPokeTester(uut) {
    poke(uut.io.in(0), 1) 
    poke(uut.io.in(1), 2)
    poke(uut.io.in(2), 8) 
    poke(uut.io.in(3), 9) 
    expect(uut.io.out, 20)
  }
}

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 10.1 ms
Total FIRRTL Compile Time: 7.6 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1532476679064
test cmd79WrapperHelperAdditiveRT Success: 1 tests passed in 5 cycles taking 0.004422 seconds
[info] [0.002] RAN 0 CYCLES PASSED


res80: Boolean = true

### Putting them Together

In [7]:
def checkparamsIPU(width: Int, bypass: String, bitwidth: Int) {
    require(width >= 1, "Width must be at least one.")
    require(List("None", "Firm").contains(bypass), "Bypass must be \"None\" or \"Firm\"")
    require(bitwidth >= 0, "Data bitwidth must be non-negative")
}


class IPU(width: Int, bypass: String, bitwidth: Int) extends Module {
    
    checkparamsIPU(width, bypass, bitwidth)
    
    val io = IO(new Bundle {
        val in1 = Input(Vec(width, SInt(bitwidth.W)))
        val in2 = Input(Vec(width, SInt(bitwidth.W)))
        val out = Output(UInt(bitwidth.W))
        val sel = if(bypass == "Firm") Some(Input(Vec(width, Bool()))) else None
        val bp1 = if(bypass == "Firm") Some(Output(UInt(bitwidth.W)))  else None
        val bp2 = if(bypass == "Firm") Some(Output(UInt(bitwidth.W)))  else None
    })
    
    val pM = new pMultiplier(width, bitwidth)
    pM.io.in1 := io.in1
    pM.io.in2 := io.in2
    
    val aRT = new AdditiveRT(width, bitwidth)
    aRT.io.in := pM.io.out
    
    io.out := aRT.io.out
    
    if (bypass == "Firm") {
        io.bp1.get := PriorityMux(io.sel.get, io.in1)
        io.bp2.get := PriorityMux(io.sel.get, io.in2)
    }
}

defined function checkparamsIPU
defined class IPU

## ALU

In [8]:
def checkparamsALU(funcs: List[String], datawidth: Int) {
    require(funcs.contains("Identity"), "ALU functions must explicitly include Identity.")
    val supportedFuncs = List("Identity", "Add", "Max", "Accumulate")
    for(x <- funcs)(require(supportedFuncs.contains(x), "Unsupported Function"))
}

class ALU(funcs: List[String], datawidth: Int) extends Module {
    
    checkparamsALU(funcs, datawidth)
 
    val io = IO(new Bundle {
        val innr_prod = Input(SInt(datawidth.W))
        val func_slct = Input(Vec(funcs.length, Bool()))
        val output    = Output(SInt(datawidth.W))
        val weight_bp = if(List("Add", "Max").contains(funcs)) Some(Input(SInt(datawidth.W))) else None
        val actvtn_bp = if(List("Add", "Max").contains(funcs)) Some(Input(SInt(datawidth.W))) else None
        val rf_feedbk = if(funcs.contains("Accumulate"))       Some(Input(SInt(datawidth.W))) else None
    })
    
    val idnOut = Some(Wire(SInt(datawidth.W)))
    val addOut = if(funcs.contains("Add"))        Some(Wire(SInt(datawidth.W))) else None
    val maxOut = if(funcs.contains("Max"))        Some(Wire(SInt(datawidth.W))) else None
    val accOut = if(funcs.contains("Accumulate")) Some(Wire(SInt(datawidth.W))) else None
    
    idnOut.get := io.innr_prod
    
    if (funcs.contains("Add")       ) { addOut.get := io.weight_bp.get + io.actvtn_bp.get }
    if (funcs.contains("Accumulate")) { accOut.get := io.innr_prod + io.rf_feedbk.get }
    if (funcs.contains("Max")       ) {
        when (io.weight_bp.get > io.weight_bp.get) {
            maxOut.get := io.weight_bp.get
        } .otherwise {
            maxOut.get := io.actvtn_bp.get
        }
    }
    
    val inters = (idnOut:: addOut :: maxOut :: accOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.output := PriorityMux(io.func_slct, inters)
}

defined function checkparamsALU
defined class ALU

## Nonlinear Unit

In [9]:
def checkparamsNLU(funcs: List[String], datawidth: Int) {
    require(funcs.contains("Identity"), "NLU functions must explicitly include Identity.")
    val supportedFuncs = List("Identity", "ReLu")
    for(x <- funcs)(require(supportedFuncs.contains(x), "Unsupported Function"))
}

class NonlinearUnit(funcs: List[String], datawidth: Int) extends Module {
    
    checkparamsNLU(funcs, datawidth)
    
    val io = IO(new Bundle {
        val input = Input(SInt(datawidth.W))
        val fslct = Input(Vec(funcs.length, Bool()))
        val outpt = Output(SInt(datawidth.W))
    })
    
    val idntOut = Some(Wire(SInt(datawidth.W)))
    val reluOut = if(funcs.contains("ReLu")) Some(Wire(SInt(datawidth.W))) else None
    
    idntOut.get := io.input
    if (funcs.contains("ReLu")) {
        when (io.input > 0.S) {
            reluOut.get := io.input
        } .otherwise {
            reluOut.get := 0.S
        }
    }
    
    val inters = (idntOut :: reluOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.outpt := PriorityMux(io.fslct, inters)
}

defined function checkparamsNLU
defined class NonlinearUnit

## State Machine

In [133]:
class StateMachine(nextState: Map[(UInt, UInt), UInt], ctrlWidth: Int) extends Module {
    
    val stateWidth: Int = log2Up(nextState.size)
    
    val io = IO(new Bundle {
        val control = Input (UInt(ctrlWidth.W ))
        val state   = Output(UInt(stateWidth.W))
    })
    
    val register = RegInit(0.U(stateWidth.W))
    register := nextState((io.state, io.control))
    io.state := register
}

defined class StateMachine

## Decoder

In [134]:
class Decoder(decode: (UInt, String) => Data,
              statewidth: Int,
              ports: Int, datawidth: Int, addrwidth: Int,
              aluFuncs: List[String], 
              nluFuncs: List[String]) extends Module {
    
    val io = IO(new Bundle {
        
        val state = Input(UInt(statewidth.W))
        
        val weightRF_wen         = Output(Vec(ports, Bool()))
        val weightRF_ren         = Output(Vec(ports, Bool()))
        val weightRF_waddr       = Output(Vec(ports, UInt(addrwidth.W)))
        val weightRF_raddr_int   = Output(Vec(ports, UInt(addrwidth.W)))
        val weightRF_raddr_ext   = Output(Vec(ports, UInt(addrwidth.W)))
        val weightRF_bp_slct_get = Output(Vec(ports, Bool()))
        
        val actvtnRF_wen         = Output(Vec(ports, Bool()))
        val actvtnRF_ren         = Output(Vec(ports, Bool()))
        val actvtnRF_waddr       = Output(Vec(ports, UInt(addrwidth)))
        val actvtnRF_raddr_int   = Output(Vec(ports, UInt(addrwidth)))
        val actvtnRF_raddr_ext   = Output(Vec(ports, UInt(addrwidth)))
        val actvtnRF_bp_slct_get = Output(Vec(ports, Bool()))
        
        val ipu_sel_get   = Output(Vec(ports, Bool()))
        
        val alu_func_slct = Output(Vec(aluFuncs.length, Bool()))
        
        val intrnlRF_write_en    = Output(Vec(ports, Bool()))
        val intrnlRF_read_en     = Output(Vec(ports, Bool()))
        val intrnlRF_waddr       = Output(Vec(ports, UInt(addrwidth)))
        val intrnlRF_raddr_int   = Output(Vec(ports, UInt(addrwidth)))
        val intrnlRF_raddr_ext   = Output(Vec(ports, UInt(addrwidth)))
        val intrnlRF_bp_slct_get = Output(Vec(ports, Bool()))
        val intrnlRF_wdata_slct  = Output(Bool())
        
        val nlu_func_slct = Output(Vec(nluFuncs.length, Bool()))
        
    })
    
    io.weightRF_wen         := decode(io.state, "weightRF_wen")
    io.weightRF_ren         := decode(io.state, "weightRF_ren")
    io.weightRF_waddr       := decode(io.state, "weightRF_waddr")
    io.weightRF_raddr_int   := decode(io.state, "weightRF_raddr_int")
    io.weightRF_raddr_ext   := decode(io.state, "weightRF_raddr_ext")
    io.weightRF_bp_slct_get := decode(io.state, "weightRF_bp_slct_get")

}

defined class Decoder

In [125]:
def decode(state: UInt, output: String): Data = {
    
    // Set Types
    val data = output match {
        case "weightRF_wen"         => Wire(Vec(2, Bool()))
        case "weightRF_ren"         => Wire(Vec(2, Bool()))
        case "weightRF_waddr"       => Wire(Vec(2, UInt(8.W)))
        case "weightRF_raddr_int"   => Wire(Vec(2, UInt(8.W)))
        case "weightRF_raddr_ext"   => Wire(Vec(2, UInt(8.W)))
        case "weightRF_bp_slct_get" => Wire(Vec(2, Bool()))
    }
    
    // Set Values
    when(state === 0.U) {
        data := { output match {
            case "weightRF_wen"         => Vec.fill(2){true.B}
            case "weightRF_ren"         => Vec.fill(2){true.B}
            case "weightRF_waddr"       => Vec.fill(2){1.U}
            case "weightRF_raddr_int"   => Vec.fill(2){2.U}
            case "weightRF_raddr_ext"   => Vec.fill(2){3.U}
            case "weightRF_bp_slct_get" => Vec.fill(2){true.B}
        }}
    } 

    .otherwise {
        data := { output match {
             case "weightRF_wen"         => Vec.fill(2){false.B}
             case "weightRF_ren"         => Vec.fill(2){false.B}
             case "weightRF_waddr"       => Vec.fill(2){4.U}
             case "weightRF_raddr_int"   => Vec.fill(2){5.U}
             case "weightRF_raddr_ext"   => Vec.fill(2){6.U}
             case "weightRF_bp_slct_get" => Vec.fill(2){false.B}
         }}
    }
    
    data
}

defined function decode

In [131]:
println(getVerilog(new Decoder(decode, statewidth=4, ports=2,
                        datawidth=4, addrwidth=4,
                        aluFuncs=List("None"),
                        nluFuncs=List("None"))))

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 21.5 ms

module cmd123WrapperHelperDecoder( // @[:@3.2]
  input        clock, // @[:@4.4]
  input        reset, // @[:@5.4]
  input  [3:0] io_state, // @[:@6.4]
  output       io_weightRF_wen_0, // @[:@6.4]
  output       io_weightRF_wen_1, // @[:@6.4]
  output       io_weightRF_ren_0, // @[:@6.4]
  output       io_weightRF_ren_1, // @[:@6.4]
  output [3:0] io_weightRF_waddr_0, // @[:@6.4]
  output [3:0] io_weightRF_waddr_1, // @[:@6.4]
  output [3:0] io_weightRF_raddr_int_0, // @[:@6.4]
  output [3:0] io_weightRF_raddr_int_1, // @[:@6.4]
  output [3:0] io_weightRF_raddr_ext_0, // @[:@6.4]
  output [3:0] io_weightRF_raddr_ext_1, // @[:@6.4]
  output       io_weightRF_bp_slct_get_0, // @[:@6.4]
  output       io_weightRF_bp_slct_get_1 // @[:@6.4]
);
  wire  _T_48_0; // @[cmd124.sc 5:44:@8.4]
  wire  _T_48_1; // @[cmd124.sc 5:44:@8.4]
  wire  _T_54; // @[cmd124.sc 14:16:@9.4]
  wire  _T_76_0;

In [130]:
Driver(() => new Decoder(decode, statewidth=4, ports=2,
                        datawidth=4, addrwidth=4,
                        aluFuncs=List("None"),
                        nluFuncs=List("None"))) {
    
    uut => new PeekPokeTester(uut) {
        
        
        poke(uut.io.state, 0.U)
        step(1)
        
        expect(uut.io.weightRF_wen(0), true.B)
        expect(uut.io.weightRF_ren(0), true.B)
        expect(uut.io.weightRF_waddr(0), 1.U)
        expect(uut.io.weightRF_raddr_int(0), 2.U)
        expect(uut.io.weightRF_raddr_ext(0), 3.U)
        expect(uut.io.weightRF_bp_slct_get(0), true.B)
        
        poke(uut.io.state, 1.U) 
        step(1)
        
        expect(uut.io.weightRF_wen(0), false.B)
        expect(uut.io.weightRF_ren(0), false.B)
        expect(uut.io.weightRF_waddr(0), 4.U)
        expect(uut.io.weightRF_raddr_int(0), 5.U)
        expect(uut.io.weightRF_raddr_ext(0), 6.U)
        expect(uut.io.weightRF_bp_slct_get(0), false.B)
        
    }
}


[info] [0.000] Elaborating design...
[info] [0.005] Done elaborating.
Total FIRRTL Compile Time: 21.9 ms
Total FIRRTL Compile Time: 16.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1532479097135
test cmd123WrapperHelperDecoder Success: 12 tests passed in 7 cycles taking 0.012984 seconds
[info] [0.009] RAN 2 CYCLES PASSED


res129: Boolean = true